In [196]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.holtwinters import SimpleExpSmoothing


In [197]:
dados = pd.read_csv('base_dados.csv')
df = pd.DataFrame(dados)

print(df.head())

   Unnamed: 0   ID-Produto             Produto         Categoria     Segmento  \
0           0  SKU-0000002    Geladeira Duplex  Eletrodomésticos    Doméstico   
1           1  SKU-0000040  Notebook Dell 8 GB       Eletrônicos  Corporativo   
2           2  SKU-0000064         Micro-Ondas  Eletrodomésticos   Industrial   
3           3  SKU-0000041  Notebook Dell 8 GB       Eletrônicos  Corporativo   
4           4  SKU-0000065               Grill  Eletrodomésticos    Doméstico   

  Fabricante    Loja          Cidade          Estado          Vendedor  \
0   Brastemp  SP8821       São Paulo       São Paulo      Josias Silva   
1       Dell   R1296  Rio de Janeiro  Rio de Janeiro      Aline Sutter   
2     Consul  SP8823       São Paulo       São Paulo  Mateus Gonçalves   
3       Dell   R1296  Rio de Janeiro  Rio de Janeiro      Ana Teixeira   
4  Panasonic  SP8823       São Paulo       São Paulo     Artur Moreira   

   ID-Vendedor  Comissão (Percentual)  Data Venda  ValorVenda  Custo

In [198]:
#Análise exploratória

#Verificando se existe valores ausentes

valores_ausentes = df.isnull().sum()
print(valores_ausentes)

colunas = df.columns
print(colunas)
print(df.shape)


Unnamed: 0               0
ID-Produto               0
Produto                  0
Categoria                0
Segmento                 0
Fabricante               0
Loja                     0
Cidade                   0
Estado                   0
Vendedor                 0
ID-Vendedor              0
Comissão (Percentual)    0
Data Venda               0
ValorVenda               0
Custo                    0
Satisfação Cliente       0
dtype: int64
Index(['Unnamed: 0', 'ID-Produto', 'Produto', 'Categoria', 'Segmento',
       'Fabricante', 'Loja', 'Cidade', 'Estado', 'Vendedor', 'ID-Vendedor',
       'Comissão (Percentual)', 'Data Venda', 'ValorVenda', 'Custo',
       'Satisfação Cliente'],
      dtype='object')
(457, 16)


In [199]:
#Não existe valores duplicados
valores_duplicados = df.duplicated()

print(valores_duplicados)

0      False
1      False
2      False
3      False
4      False
       ...  
452    False
453    False
454    False
455    False
456    False
Length: 457, dtype: bool


In [200]:
#Verificação dos tipos dos dados
print(df.dtypes)

Unnamed: 0                 int64
ID-Produto                object
Produto                   object
Categoria                 object
Segmento                  object
Fabricante                object
Loja                      object
Cidade                    object
Estado                    object
Vendedor                  object
ID-Vendedor                int64
Comissão (Percentual)      int64
Data Venda                object
ValorVenda               float64
Custo                    float64
Satisfação Cliente         int64
dtype: object


In [201]:
#transformando coluna Data Vendas do tipo Data

df['Data Venda'] = pd.to_datetime(df['Data Venda'])

print(df.dtypes)

Unnamed: 0                        int64
ID-Produto                       object
Produto                          object
Categoria                        object
Segmento                         object
Fabricante                       object
Loja                             object
Cidade                           object
Estado                           object
Vendedor                         object
ID-Vendedor                       int64
Comissão (Percentual)             int64
Data Venda               datetime64[ns]
ValorVenda                      float64
Custo                           float64
Satisfação Cliente                int64
dtype: object


In [202]:
#Distribuição dos dados
#resumo estatístico
descricao = df.describe().round(2)

print(descricao)

       Unnamed: 0  ID-Vendedor  Comissão (Percentual)  \
count      457.00       457.00                 457.00   
mean       228.00      1004.09                   2.14   
min          0.00      1001.00                   2.00   
25%        114.00      1002.00                   2.00   
50%        228.00      1004.00                   2.00   
75%        342.00      1006.00                   2.00   
max        456.00      1009.00                   6.00   
std        132.07         2.46                   0.57   

                          Data Venda  ValorVenda   Custo  Satisfação Cliente  
count                            457      457.00  457.00              457.00  
mean   2014-07-15 22:28:37.286652160      786.23  132.75                3.07  
min              2012-01-02 00:00:00       12.00   90.00                1.00  
25%              2013-08-02 00:00:00      167.00  120.00                2.00  
50%              2014-12-04 00:00:00      671.00  120.00                3.00  
75%         

In [203]:
#verificando tabela valor venda e custo

coluna_valor_venda = df['ValorVenda']
coluna_valor_custo = df['Custo']

print(coluna_valor_venda)
print(coluna_valor_custo)

0       832.00
1      1345.87
2       459.00
3      1234.12
4       234.00
        ...   
452     455.00
453     149.00
454     121.00
455     346.00
456     134.00
Name: ValorVenda, Length: 457, dtype: float64
0      712.0
1      120.0
2      120.0
3      120.0
4      120.0
       ...  
452    120.0
453    120.0
454    120.0
455    120.0
456    120.0
Name: Custo, Length: 457, dtype: float64


In [204]:
'''df['ValorVenda'] = (
    df['ValorVenda']
    .str.strip() 
    .str.replace('$', '', regex=False)
    .str.replace('.', '', regex=False)  # Remove separadores de milhar
    .str.replace(',', '.', regex=False)  # Substitui a vírgula por ponto
)
'''

print(df['ValorVenda'])

0       832.00
1      1345.87
2       459.00
3      1234.12
4       234.00
        ...   
452     455.00
453     149.00
454     121.00
455     346.00
456     134.00
Name: ValorVenda, Length: 457, dtype: float64


In [205]:
df['ValorVenda'] = df['ValorVenda'].astype(float).map('{:.2f}'.format)

print(df['ValorVenda'].dtype)




object


In [206]:
print(df['ValorVenda'])

0       832.00
1      1345.87
2       459.00
3      1234.12
4       234.00
        ...   
452     455.00
453     149.00
454     121.00
455     346.00
456     134.00
Name: ValorVenda, Length: 457, dtype: object


In [207]:
'''

df['Custo'] = (
    df['Custo']
    .str.strip()  # Remove espaços em branco
    .str.replace('$', '', regex=False)  # Remove o símbolo de dólar
    .str.replace('.', '', regex=False)  # Remove separadores de milhar, se houver
    .str.replace(',', '.', regex=False)  # Substitui a vírgula por ponto
).astype(float)
'''


"\n\ndf['Custo'] = (\n    df['Custo']\n    .str.strip()  # Remove espaços em branco\n    .str.replace('$', '', regex=False)  # Remove o símbolo de dólar\n    .str.replace('.', '', regex=False)  # Remove separadores de milhar, se houver\n    .str.replace(',', '.', regex=False)  # Substitui a vírgula por ponto\n).astype(float)\n"

In [208]:
# Converter para float
df['Custo'] = df['Custo'].astype(float)

print(df['Custo'])

0      712.0
1      120.0
2      120.0
3      120.0
4      120.0
       ...  
452    120.0
453    120.0
454    120.0
455    120.0
456    120.0
Name: Custo, Length: 457, dtype: float64


In [209]:
#df['Liquido'] = df['ValorVenda'] - df['Custo']
#df['Liquido'] = df['Liquido'].round(2)

In [210]:
#gera_arquivo = df.to_csv('base_dados.csv')

In [213]:
print(df.dtypes)

Unnamed: 0                        int64
ID-Produto                       object
Produto                          object
Categoria                        object
Segmento                         object
Fabricante                       object
Loja                             object
Cidade                           object
Estado                           object
Vendedor                         object
ID-Vendedor                       int64
Comissão (Percentual)             int64
Data Venda               datetime64[ns]
ValorVenda                       object
Custo                           float64
Satisfação Cliente                int64
dtype: object


In [211]:
#Trabalhando com comissão

def comissao(valor):
    comissoes = {
        2: 0.5,
        3: 0.10,
        4: 0.15,
        5: 0.20,
        6: 0.25
    }
    return valor + comissoes.get(valor, 0)  # Retorna 0 se valor não estiver no dicionário

In [212]:
df['Valor_Comissao'] = df['ValorVenda'].apply(comissao)

TypeError: can only concatenate str (not "int") to str

In [ ]:
print(df['Valor_Comissao'])